#**Amazon Book Reviews**

El dataset a utilizar durante el proyecto es ‘Amazon Book Reviews’ proveniente de la
plataforma Kaggle, se encuentra disponible en el siguiente enlace:
https://www.kaggle.com/datasets/mohamedbakhet/amazon-books-reviews/data.

Este conjunto de datos fue construido a partir de dos fuentes principales: la primera
fuente es el repositorio de datos de Amazon de la Universidad de California en San
Diego (UCSD),  La segunda fuente corresponde a datos obtenidos a través de la
API de Google Books.

En cuanto al tamaño global, el dataset se compone de dos archivos principales:
books_data.csv, con un tamaño aproximado de 181.3 MB, y books_rating.csv, con un
tamaño aproximado de 2.86 GB

Los datasets books_data y books_rating ofrecen información complementaria sobre
libros disponibles en Amazon y las reseñas realizadas por los usuarios. A continuación
se presenta una descripción general de cada uno de los datasets:

**books_data.csv (Metadatos de los libros):**
Este dataset contiene metadatos como título, autor, categoría, número de valoraciones,
descripción, fecha de publicación. En cuanto a su estructura cuenta con 212,404
registros y 10 columnas.

**books_rating.csv (Reseñas de Usuarios):** Este dataset tiene un enfoque en las opiniones de los usuarios que incluyen:
puntuaciones, reseñas, ID de usuario y nombre de perfil. En cuanto a su estructura
cuenta con 3,000,000 de registros distribuidos en 10 columnas


# **Proyecto | lectura, escritura, archivos de Big Data PySpark**

## **CARGA DE ARCHIVOS**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when, to_date
from pyspark.sql.types import DoubleType

spark = SparkSession.builder.appName("AmazonBooksEDA").getOrCreate()

# Carga de archivos
books_df = spark.read.option("header", True).option("inferSchema", True).csv("data/raw/books_data.csv")
ratings_df = spark.read.option("header", True).option("inferSchema", True).csv("data/raw/Books_rating.csv")



In [ ]:
print("books_data.csv:")
books_df.show(5)
books_df.printSchema()

print("\nBooks_rating.csv:")
ratings_df.show(5)
ratings_df.printSchema()


books_data.csv:
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|               Title|         description|             authors|               image|         previewLink|           publisher| publishedDate|            infoLink|          categories|ratingsCount|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+------------+
|Its Only Art If I...|                NULL|    ['Julie Strain']|http://books.goog...|http://books.goog...|                NULL|          1996|http://books.goog...|['Comics & Graphi...|        NULL|
|Dr. Seuss: Americ...|"Philip Nel takes...| like that of Lew...| has changed lang...| giving us new wo...| inspiring artist...|['Philip Nel']|http://books.goog...|http://books.goog...|   A&C B

In [ ]:
books_df = books_df.withColumn("publishedDate", to_date(col("publishedDate"))) \
                   .withColumn("ratingsCount", col("ratingsCount").cast("integer"))

ratings_df = ratings_df.withColumn("review/score", col("review/score").cast(DoubleType()))

print("books_data.csv:")
books_df.printSchema()

print("\nBooks_rating.csv:")
ratings_df.printSchema()

books_data.csv:
root
 |-- Title: string (nullable = true)
 |-- description: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- image: string (nullable = true)
 |-- previewLink: string (nullable = true)
 |-- publisher: string (nullable = true)
 |-- publishedDate: date (nullable = true)
 |-- infoLink: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- ratingsCount: integer (nullable = true)


Books_rating.csv:
root
 |-- Id: string (nullable = true)
 |-- Title: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- User_id: string (nullable = true)
 |-- profileName: string (nullable = true)
 |-- review/helpfulness: string (nullable = true)
 |-- review/score: double (nullable = true)
 |-- review/time: string (nullable = true)
 |-- review/summary: string (nullable = true)
 |-- review/text: string (nullable = true)



In [ ]:
print("books_data.csv - filas:", books_df.count(), "columnas:", len(books_df.columns))
print("Books_rating.csv - filas:", ratings_df.count(), "columnas:", len(ratings_df.columns))


books_data.csv - filas: 212404 columnas: 10


[Stage 54:===========================================>            (17 + 5) / 22]

Books_rating.csv - filas: 3000000 columnas: 10


In [ ]:
print("Valores nulos en books_data.csv:")
books_df.select([count(when(col(c).isNull(), c)).alias(c) for c in books_df.columns]).show()

print("Valores nulos en Books_rating.csv:")
ratings_df.select([count(when(col(c).isNull(), c)).alias(c) for c in ratings_df.columns]).show()



Valores nulos en books_data.csv:
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+
|Title|description|authors|image|previewLink|publisher|publishedDate|infoLink|categories|ratingsCount|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+
|    1|      68357|  31251|51191|      24055|    73130|        46999|   24301|     40524|      168972|
+-----+-----------+-------+-----+-----------+---------+-------------+--------+----------+------------+

Valores nulos en Books_rating.csv:


[Stage 60:=============================================>          (18 + 4) / 22]

+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+
| Id|Title|  Price|User_id|profileName|review/helpfulness|review/score|review/time|review/summary|review/text|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+
|  0|  208|2517579| 562250|     562200|               367|       18064|         27|            65|         43|
+---+-----+-------+-------+-----------+------------------+------------+-----------+--------------+-----------+



In [ ]:
print("Estadísticas books_data.csv:")
books_df.describe().show()

print("Ratings únicos en books_data.csv:")
books_df.select("ratingsCount").distinct().show()

print("Estadísticas Books_rating.csv:")
ratings_df.describe().show()


Estadísticas books_data.csv:


+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|               Title|         description|             authors|               image|         previewLink|           publisher|            infoLink|          categories|      ratingsCount|
+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+
|  count|              212403|              144047|              181153|              161213|              188349|              139274|              188103|              171880|             43432|
|   mean|   3823.672941176471|  1.4285714285714286|              1578.4|              1184.0|            Infinity|             3734.75|   1989.054693877551|  1983.7334777898159| 56.31520537852275|
| stddev|  1071

[Stage 69:===========================================>            (17 + 5) / 22]

+-------+--------------------+--------------------+--------------------+-------------------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+
|summary|                  Id|               Title|               Price|            User_id|profileName| review/helpfulness|      review/score|         review/time|      review/summary|         review/text|
+-------+--------------------+--------------------+--------------------+-------------------+-----------+-------------------+------------------+--------------------+--------------------+--------------------+
|  count|             3000000|             2999792|              482421|            2437750|    2437800|            2999633|           2981936|             2999973|             2999935|             2999957|
|   mean|1.0568515696607149E9|   2012.796651763537|  21.767951161877054|  18.29299003322259|        NaN|3.285048033703448E8| 1656.860421970827|1.1270533345949814E9|        

# **Proyecto | Base de Datos de Big Data**

## **PARTICIONAMIENTO**

In [ ]:
from pyspark.sql.functions import col

# Asegúrate de filtrar registros que no tengan nulos en estas dos columnas
df_clean = books_df.join(ratings_df, on="Title") \
    .filter((col("ratingsCount").isNotNull()) & (col("review/score").isNotNull()))


In [ ]:
total = df_clean.count()
print(total)


[Stage 73:=====================================================>  (21 + 1) / 22]

1413821


In [ ]:
df_clean.select("ratingsCount","review/score").describe().show()

[Stage 82:===========================================>            (17 + 5) / 22]

+-------+-----------------+------------------+
|summary|     ratingsCount|      review/score|
+-------+-----------------+------------------+
|  count|          1413821|           1413821|
|   mean|302.0177313818369| 4.214017191709559|
| stddev|814.8520705691591|1.2886346716959942|
|    min|                1|               1.0|
|    max|             4572|             327.0|
+-------+-----------------+------------------+



In [ ]:
total = df_clean.count()

# A: ratingsCount (popularidad)
p_a = df_clean.filter(col("ratingsCount") <= 1000).count() / total
p_b = 1 - p_a
p_c = df_clean.filter(col("review/score") < 4.0).count() / total
p_d = df_clean.filter(col("review/score") >= 4.0).count() / total

# B: review/score (percepción)
p_c = df_clean.filter(col("review/score") < 4.0).count() / total
p_d = 1 - p_c

In [ ]:
#Probabilidades de Ocurrencia
print("A. a:", p_a) # Probabilidad de a (ratingsCount <= 1000)
print("B. b:", p_b) # Probabilidad de b (ratingsCount > 1000)
print("C. c:", p_c) # Probabilidad de c (review/score < 4.0)
print("D. d:", p_d) # Probabilidad de d (review/score >= 4.0)

A. a: 0.9011897545728915
B. b: 0.09881024542710848
C. c: 0.20372310214659423
D. d: 0.7962768978534058


In [ ]:
# Probabilidades conjuntas
p_ac = p_a * p_c  # Probabilidad de a ∧ c (ratingsCount <= 1000 y review/score < 4.0)
p_ad = p_a * p_d  # Probabilidad de a ∧ d (ratingsCount <= 1000 y review/score >= 4.0)
p_bc = p_b * p_c  # Probabilidad de b ∧ c (ratingsCount > 1000 y review/score < 4.0)
p_bd = p_b * p_d  # Probabilidad de b ∧ d (ratingsCount > 1000 y review/score >= 4.0)


In [ ]:
#Probabilidades de Ocurrencia
print("A. a ∧ c:", p_ac)
print("B. a ∧ d:", p_ad)
print("C. b ∧ c:", p_bc)
print("D. b ∧ d:", p_bd)


A. a ∧ c: 0.18359317242431736
B. a ∧ d: 0.7175965821485742
C. b ∧ c: 0.020129929722276865
D. b ∧ d: 0.07868031570483162


## **Extracción de submuestras a partir de las reglas de particionamiento generadas**

Combinación A → ratingsCount ≤ 1000 y score < 4.0

In [ ]:
df_A = df_clean.filter((col("ratingsCount") <= 1000) & (col("review/score") < 4.0))
df_A.show(5)


[Stage 145:===============================================>       (19 + 3) / 22]

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+----------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|ratingsCount|        Id|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+----------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------

Combinación B → ratingsCount ≤ 1000 y score ≥ 4.0

In [ ]:
df_B = df_clean.filter((col("ratingsCount") <= 1000) & (col("review/score") >= 4.0))
df_B.show(5)


[Stage 150:=============================================>         (18 + 4) / 22]

+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+--------------------+-------------+------------+----------+-----+--------------+-------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|         description|             authors|               image|         previewLink|          publisher|publishedDate|            infoLink|   categories|ratingsCount|        Id|Price|       User_id|  profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------+--------------------+-------------+------------+----------+-----+--------------+-------------+------------------+------------+-----------+--------------------+--------------------+
|"Discovery of the...|This is the acc

 Combinación C → ratingsCount > 1000 y score < 4.0

In [ ]:
df_C = df_clean.filter((col("ratingsCount") > 1000) & (col("review/score") < 4.0))
df_C.show(5)


[Stage 155:=============================================>         (18 + 4) / 22]

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+----------+------+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|         description|             authors|               image|         previewLink|           publisher|publishedDate|            infoLink|          categories|ratingsCount|        Id| Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+--------------------+------------+----------+------+--------------+--------------------+------------------+------------+-----------+--------------------+-----------

Combinación D → ratingsCount > 1000 y score ≥ 4.0

In [ ]:
df_D = df_clean.filter((col("ratingsCount") > 1000) & (col("review/score") >= 4.0))
df_D.show(5)


[Stage 161:==========================================>            (17 + 5) / 22]

+--------------------+--------------------+--------------------+-----------+-----------------+--------------------+-------------+--------------------+----------+------------+----------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|               Title|         description|             authors|      image|      previewLink|           publisher|publishedDate|            infoLink|categories|ratingsCount|        Id|Price|       User_id|         profileName|review/helpfulness|review/score|review/time|      review/summary|         review/text|
+--------------------+--------------------+--------------------+-----------+-----------------+--------------------+-------------+--------------------+----------+------------+----------+-----+--------------+--------------------+------------------+------------+-----------+--------------------+--------------------+
|Raymond Chandler:...|"Later Novels and...|"" in which Cha